In [33]:
import os

In [34]:
%pwd

'c:\\Users\\aditi\\OneDrive\\Desktop\\Chest-Cancer-ML-OPS'

In [35]:
os.chdir(path='c:\\Users\\aditi\\OneDrive\\Desktop\\Chest-Cancer-ML-OPS')

In [36]:
%pwd

'c:\\Users\\aditi\\OneDrive\\Desktop\\Chest-Cancer-ML-OPS'

In [37]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen = True)
class PrepareBaseModelConfig:
    root_dir : Path
    base_model_path : Path
    updated_base_model_path : Path
    image_size_params : list
    learning_rate_params : float
    include_top_params : bool
    weights_params : str
    classes_params : int
    

In [38]:
from chest_cancer_classifier.constants import *
from chest_cancer_classifier.utils.common import read_yaml, create_dir

In [39]:
class ConfigManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        param_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(param_filepath)
        
        create_dir([self.config.artifacts_root])
        
    def prepare_base_model(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model
        create_dir([config.root_dir])
        
        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir= Path(config.root_dir),
            base_model_path = Path(config.base_model_path),
            updated_base_model_path = Path(config.updated_base_model_path),
            image_size_params = self.params.IMAGE_SIZE,
            learning_rate_params=self.params.LEARNING_RATE,
            include_top_params=self.params.INCLUDE_TOP,
            weights_params=self.params.WEIGHTS,
            classes_params=self.params.CLASSES 
        )
        return prepare_base_model_config

In [40]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf

In [41]:
class PrepareBaseModel:
    def __init__(self,config : PrepareBaseModelConfig):
        self.config = config
        
    def get_base_model(self):
        self.model = tf.keras.applications.vgg16.VGG16(
            input_shape=self.config.image_size_params,
            weights=self.config.weights_params,
            include_top=self.config.include_top_params
        )
        
        self.save_model(path = self.config.base_model_path,model= self.model)
        
    @staticmethod
    def prepare_full_model(model,classes,freeze_all,freeze_till,learning_rate):
        if freeze_all:
            for layer in model.layers:
                model.trainable = False
        elif freeze_till is not None and freeze_till > 0 :
            for layer in model.layers[:-freeze_till]:
                model.trainable = False
                
        flatten_in = tf.keras.layers.Flatten()(model.output)
        prediction = tf.keras.layers.Dense(
            units=classes,
            activation="softmax"
        )(flatten_in)    
        
        full_model = tf.keras.models.Model(
            inputs = model.input,
            outputs = prediction
        )    
        
        full_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=['accuracy']
        )
        
        full_model.summary()
        return full_model
    
    def update_base_model(self):
        self.full_model = self.prepare_full_model(
            model = self.model,
            classes = self.config.classes_params,
            freeze_all = True,
            freeze_till = None,
            learning_rate = self.config.learning_rate_params
        )
        
        self.save_model(path = self.config.updated_base_model_path,model=self.full_model)
        
        
    @staticmethod
    def save_model(path:Path , model:tf.keras.Model):
        model.save(path)

In [42]:
try:
    config = ConfigManager()
    prepare_base_model_config = config.prepare_base_model()
    prepare_base_model = PrepareBaseModel(config = prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
    
except Exception as e:
    raise e
    

path C:\Users\aditi\OneDrive\Desktop\Chest-Cancer-ML-OPS\config\config.yaml
[2024-08-31 18:57:51,412 INFO : common : YAML file C:\Users\aditi\OneDrive\Desktop\Chest-Cancer-ML-OPS\config\config.yaml successfully loaded]
path C:\Users\aditi\OneDrive\Desktop\Chest-Cancer-ML-OPS\params.yaml
[2024-08-31 18:57:51,417 INFO : common : YAML file C:\Users\aditi\OneDrive\Desktop\Chest-Cancer-ML-OPS\params.yaml successfully loaded]
[2024-08-31 18:57:51,419 INFO : common : Created Directory at : artifacts]
[2024-08-31 18:57:51,421 INFO : common : Created Directory at : artifacts/prepare_base_model]
[2024-08-31 18:57:52,170 WARNING : saving_utils : Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0  